# Predict customer retention by deep learning

## Dataset

In [77]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import model_selection,pipeline,compose,preprocessing
from tensorflow import feature_column
from tensorflow.keras import layers
# OOP python for feature engineering
from sklearn.base import BaseEstimator, TransformerMixin

In [59]:
path='https://raw.githubusercontent.com/rstudio/keras-customer-churn/master/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'
dataframe=pd.read_csv(path)
dataframe.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [60]:
dataframe.shape

(7043, 21)

In [61]:
dataframe.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


## Create target variable

In [62]:
# In the original dataset "4" indicates the pet was not adopted. ( No need in this case)
#dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)
dataframe.rename(columns = {'Churn': 'target'}, inplace=True)
# Drop un-used columns.
dataframe = dataframe.drop(columns=['customerID'])
dataframe.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,target
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [63]:
train, test = model_selection.train_test_split(dataframe, test_size=0.2)
train, val = model_selection.train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

4507 train examples
1127 validation examples
1409 test examples


Next, we will wrap the dataframes with tf.data. This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [64]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [65]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size) # need suffle train ?
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [68]:
print(len(list(train_ds))) #902 batchs, each batch 5 sample --> 4507 sample
print(len(list(val_ds)))
print(len(list(test_ds)))

902
226
282


In [70]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of MonthlyCharges:', train_features['MonthlyCharges'])
print('A batch of targets:', label_batch )

Every feature: ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
A batch of MonthlyCharges: tf.Tensor([89.   85.45 67.45 89.8  81.1 ], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([b'No' b'Yes' b'No' b'Yes' b'Yes'], shape=(5,), dtype=string)


Demonstrate several types of feature columns

In [72]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]
example_batch # contain only x train, not include y train

{'Contract': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'One year', b'Month-to-month', b'One year', b'Two year',
        b'One year'], dtype=object)>,
 'Dependents': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Yes', b'No', b'No', b'No', b'No'], dtype=object)>,
 'DeviceProtection': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Yes', b'No', b'No', b'No internet service', b'No'], dtype=object)>,
 'InternetService': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'DSL', b'Fiber optic', b'Fiber optic', b'No', b'Fiber optic'],
       dtype=object)>,
 'MonthlyCharges': <tf.Tensor: shape=(5,), dtype=float64, numpy=array([44.25, 94.55, 90.45, 25.05, 93.55])>,
 'MultipleLines': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'No phone service', b'Yes', b'Yes', b'Yes', b'No'], dtype=object)>,
 'OnlineBackup': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'No', b'No', b'Yes', b'No internet service', b'Yes'], dtype=object)>,
 'OnlineSecurity': <tf.Tensor: 

In [76]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

### Numeric columns
The output of a feature column becomes the input to the model (using the demo function defined above, we will be able to see exactly how each column from the dataframe is transformed). A [numeric column](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) is the simplest type of column. It is used to represent real valued features. When using this column, your model will receive the column value from the dataframe unchanged.

In [74]:
example_batch['MonthlyCharges']

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([44.25, 94.55, 90.45, 25.05, 93.55])>

In [78]:
photo_count = feature_column.numeric_column('MonthlyCharges') # feature column from tensorflow
photo_count

NumericColumn(key='MonthlyCharges', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

In [79]:
demo(photo_count) # show one sample from feature column

[[44.25]
 [94.55]
 [90.45]
 [25.05]
 [93.55]]


### Bucketized columns
Often, you don't want to feed a number directly into the model, but instead split its value into different categories based on numerical ranges. Consider raw data that represents a person's age. Instead of representing age as a numeric column, we could split the age into several buckets using a [bucketized column](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column). Notice the one-hot values below describe which age range each row matches.

In [80]:
example_batch['MonthlyCharges']

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([44.25, 94.55, 90.45, 25.05, 93.55])>

In [84]:
age = feature_column.numeric_column('MonthlyCharges')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 50, 200])
age_buckets

BucketizedColumn(source_column=NumericColumn(key='MonthlyCharges', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(1, 50, 200))

In [85]:
demo(age_buckets)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


In [83]:
age_buckets = feature_column.bucketized_column(age, boundaries=[10, 50, 100])
demo(age_buckets)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]


### Categorical columns
In this dataset, Type is represented as a string (e.g. 'Dog', or 'Cat'). We cannot feed strings directly to a model. Instead, we must first map them to numeric values. The categorical vocabulary columns provide a way to represent strings as a one-hot vector (much like you have seen above with age buckets). The vocabulary can be passed as a list using [categorical_column_with_vocabulary_list](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list), or loaded from a file using [categorical_column_with_vocabulary_file](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_file).

In [86]:
example_batch['PhoneService']

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'No', b'Yes', b'Yes', b'Yes', b'Yes'], dtype=object)>

In [95]:
animal_type = feature_column.categorical_column_with_vocabulary_list(
      'PhoneService', dataframe.PhoneService.unique())

animal_type

VocabularyListCategoricalColumn(key='PhoneService', vocabulary_list=('No', 'Yes'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [97]:
animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


### Embedding columns
Suppose instead of having just a few possible strings, we have thousands (or more) values per category. For a number of reasons, as the number of categories grow large, it becomes infeasible to train a neural network using one-hot encodings. We can use an embedding column to overcome this limitation. Instead of representing the data as a one-hot vector of many dimensions, an [embedding column](https://www.tensorflow.org/api_docs/python/tf/feature_column/embedding_column) represents that data as a lower-dimensional, dense vector in which each cell can contain any number, not just 0 or 1. The size of the embedding (8, in the example below) is a parameter that must be tuned.

Key point: using an embedding column is best when a categorical column has many possible values. We are using one here for demonstration purposes, so you have a complete example you can modify for a different dataset in the future.

In [92]:
example_batch['MultipleLines']

<tf.Tensor: shape=(5,), dtype=string, numpy=array([b'No phone service', b'Yes', b'Yes', b'Yes', b'No'], dtype=object)>

In [98]:
# Notice the input to the embedding column is the categorical column
# we previously created
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'MultipleLines', dataframe.MultipleLines.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=4)
demo(breed1_embedding)

[[ 0.12943156  0.09426557 -0.13599777 -0.5696442 ]
 [-0.2687577   0.06196275 -0.38225695 -0.23939762]
 [-0.2687577   0.06196275 -0.38225695 -0.23939762]
 [-0.2687577   0.06196275 -0.38225695 -0.23939762]
 [ 0.5436325   0.18525265 -0.08836998  0.44740388]]


### Hashed feature columns

Another way to represent a categorical column with a large number of values is to use a [categorical_column_with_hash_bucket](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_hash_bucket). This feature column calculates a hash value of the input, then selects one of the `hash_bucket_size` buckets to encode a string. When using this column, you do not need to provide the vocabulary, and you can choose to make the number of hash_buckets significantly smaller than the number of actual categories to save space.

Key point: An important downside of this technique is that there may be collisions in which different strings are mapped to the same bucket. In practice, this can work well for some datasets regardless.

In [99]:
breed1_hashed = feature_column.categorical_column_with_hash_bucket(
      'MultipleLines', hash_bucket_size=10)
demo(feature_column.indicator_column(breed1_hashed))

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
